In [1]:
!export HF_ENDPOINT=https://hf-mirror.com

In [ ]:
!huggingface-cli download --resume-download Geotrend/bert-base-en-zh-cased --local-dir /home/RE/LitTrain/PreModel/Bert-zh-en-cased --local-dir-use-symlinks False

In [15]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/home/RE/LitTrain/PreModel/Bert-zh-en-cased")
model = AutoModel.from_pretrained("/home/RE/LitTrain/PreModel/Bert-zh-en-cased")


Some weights of BertModel were not initialized from the model checkpoint at /home/RE/LitTrain/PreModel/Bert-zh-en-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Load_datasets


In [16]:
import json 

relation = ['attributed-to','exists','indicates','related-to','targets','uses','owns','located','discory','happen','mitigates','through','host']
entity = ['idt','mlw','at','ta','vul','cof','ass','ioc','cam','tool','time','trans','loc','moti',]
def extract(line):
    triples = line['triples'].split(',')
    stack = []
    datalins = []
    targets = []
    for itm in triples:
        
        its = itm.split(' ')
        dc = {}
        sign=0
        for it in its:
            
            if it.lower() in entity:
                if sign ==0:
                    
                    dc['subject'] = ' '.join(stack)
                    stack =[]
                    dc['subject_type'] = it.upper()
                    sign = 1
                else:
                    dc['object']=' '.join(stack)
                    stack = []
                    dc['object_type'] = it.upper()
                    
            elif it.lower() in relation and sign ==1:
                dc['relation']=it.lower()
                break
            else:
                stack.append(it)
        datalins.append(dc)
    targets=set([(dc['subject'],dc['object'],dc['relation']) for dc in datalins])
        
    return {"inputs":line['context'], "label":datalins, 'targets':targets}

# CTIdataset.map(extract)

                

In [17]:
with open('/home/RE/Dataset/TK.json','r',encoding='utf-8')as f1:
    listx = json.load(f1)
with open('/home/RE/Dataset/dealed31.json','r',encoding='utf-8')as f1:
    listx.extend(json.load(f1))
from datasets import Dataset
CTIdataset = Dataset.from_list(listx)
print(CTIdataset)

Dataset({
    features: ['context', 'triples'],
    num_rows: 741
})


In [18]:
targetdataset=CTIdataset.map(extract)

Map: 100%|██████████| 741/741 [00:00<00:00, 2568.03 examples/s]


In [19]:
targetdataset=targetdataset.remove_columns(['context','triples'])

In [20]:
targetdataset

Dataset({
    features: ['inputs', 'label', 'targets'],
    num_rows: 741
})

In [22]:
targetdataset[0]

{'inputs': '研究人员发现LOBSHOT是通过Windows系统的BITS服务进行恶意代码下载和执行的后门程序。BITS服务是一种文件传输技术，常用于Windows更新。LOBSHOT利用这个服务绕过防火墙和杀毒软件，实现隐蔽攻击。研究人员对LOBSHOT的机制和通信协议进行了分析，并提供了检测规则和防护建议。',
 'label': [{'object': 'BITS服务',
   'object_type': 'TOOL',
   'relation': 'uses',
   'subject': 'LOBSHOT',
   'subject_type': 'MLW'},
  {'object': 'BITS服务',
   'object_type': 'TOOL',
   'relation': 'owns',
   'subject': 'Windows系统',
   'subject_type': 'ASS'},
  {'object': '恶意代码下载和执行',
   'object_type': 'AT',
   'relation': 'uses',
   'subject': 'LOBSHOT',
   'subject_type': 'MLW'}],
 'targets': [['LOBSHOT', '恶意代码下载和执行', 'uses'],
  ['LOBSHOT', 'BITS服务', 'uses'],
  ['Windows系统', 'BITS服务', 'owns']]}

In [7]:
opt={"subj_type2id":{itm.upper():ith for ith,itm in enumerate(entity,start=1)},"predicate2id":{itm.lower():ith for ith,itm in enumerate(relation,start=1)}}

In [8]:
opt

{'subj_type2id': {'IDT': 1,
  'MLW': 2,
  'AT': 3,
  'TA': 4,
  'VUL': 5,
  'COF': 6,
  'ASS': 7,
  'IOC': 8,
  'CAM': 9,
  'TOOL': 10,
  'TIME': 11,
  'TRANS': 12,
  'LOC': 13,
  'MOTI': 14},
 'predicate2id': {'attributed-to': 1,
  'exists': 2,
  'indicates': 3,
  'related-to': 4,
  'targets': 5,
  'uses': 6,
  'owns': 7,
  'located': 8,
  'discory': 9,
  'happen': 10,
  'mitigates': 11,
  'through': 12,
  'host': 13}}

### 在本地加载Json数据

In [1]:
from typing import Literal, Union
import json, codecs
from datasets import Dataset, IterableDataset, load_dataset
def dataset_loader(path :str,type:Literal['json','arrow'], stream:bool) -> Union[Dataset,IterableDataset]:
    if type == 'json':
        train = json.load(codecs.open(path,'r',encoding='utf-8'))
    elif type == "arrow":
        pass
    else:
        raise TypeError
    
    dataset = Dataset.from_list(train)
    if stream:
        dataset.to_iterable_dataset()
    return dataset

from transformers import AutoTokenizer
dataset = dataset_loader('/home/RE/Dataset/CTI.json',type='json',stream=True)


/root/anaconda3/envs/cypy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
dataset

Dataset({
    features: ['input', 'output', 'prompt'],
    num_rows: 767
})

### 处理dataset

源格式 {input,[{relationlist}]}


{inputs context
labels [

    subj1:{subj1,objs,rela},
    subj2:{subj2,objs,rela},
    ...
]
}

1. 加载分词器

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/RE/LitTrain/PreModel/Bert-zh-en-cased")

In [2]:
import json, os
from collections import defaultdict
from typing import Dict, List
def return_posi(list1:List,list2: List):
    '''
    return the start position and end position of list2 in the list1
    '''
    itm = list2[1:-1]
    try:
        
        eco = list1.index(itm[0])
    except Exception as e:
        print(f"{list1}")
        print(f"{itm} is not exist.detail {e}")
        raise e
    ovo = list1.index(itm[-1])
    if len(itm)==1:
        return eco,eco+1
    if list1[eco:ovo+1]==itm:
        return eco,ovo+1
    else:
        
        fal={"head":0,"rear":0}# 记录开始和结束下标
        for im, j in enumerate(range(eco,len(list1)),start=eco):
            if list1[im] == itm[0] and fal['head']==0:
                fal['head']=im
                
                for b,x in enumerate(itm[1:],start=im+1):
                    # 如果当前值相同
                    if x ==list1[b]:#
                        fal['rear']=b
                        continue
                    else:
                        fal['head']=0
                        fal['rear']=0 
                        break

                if x==itm[-1] and fal['head']!=0:
                    fal['rear']=b
                    
    if list1[fal["head"]:fal['rear']+1]==itm:
        return fal["head"],fal['rear']+1
    else:
        print(itm)
        raise "ERROR"
        return 0,0  
def Process_to_pickle(oner:Dict, tokenizer,opt)->Dict:
    # print(oner)
    '''
        oner=['inputs','label']--> inputs -subjoutput[] 
                                                ---subjoutput[0] objoutput[]
            listone [subj,obj,relation],[]
            
    '''

    txt=tokenizer.encode(oner["inputs"])
    listone=defaultdict(list)

    for item in oner["label"]:

        em1=tokenizer.encode(item['subject'])
        em2=tokenizer.encode(item['object'])
        eca_1,out_1 = return_posi(txt,em1)
        e1_type_id = opt["subj_type2id"][item['subject_type']]#获取实体1的类型
        eca_2,out_2 = return_posi(txt,em2)
        o1_type_id = opt["subj_type2id"][item['object_type']]
        rela = opt['predicate2id'][item['relation'].lower()]
        if not( txt[eca_1:out_1] == em1[1:-1] and em2[1:-1] == txt[eca_2:out_2]):
            raise "errors 提取错误"
            
        # print({str((eca_1,out_1+1)):e1_type_id,str((eca_2,out_2+1)):o1_type_id,"relaid":rela})
        
        listone[str((eca_1,out_1))].append({str((eca_1,out_1)):e1_type_id,str((eca_2,out_2)):o1_type_id,"relaid":rela})
        
        '''
        combine list
        '''
    return {"inputs":oner['inputs'],"labels":json.dumps(listone,ensure_ascii=False)}
    

In [23]:
from functools import partial
cusprocess = partial(Process_to_pickle,tokenizer=tokenizer,opt=opt)

REdataset = targetdataset.map(cusprocess)


Map: 100%|██████████| 741/741 [00:00<00:00, 1759.64 examples/s]


In [62]:
REdataset

Dataset({
    features: ['inputs', 'label', 'targets', 'labels'],
    num_rows: 741
})

In [24]:
REdataset = REdataset.remove_columns(['label'])

In [64]:
REdataset

Dataset({
    features: ['inputs', 'targets', 'labels'],
    num_rows: 741
})

1. 检查REdataset是否有重复标记

In [25]:
for th,i in enumerate(REdataset):
    labels = json.loads(i['labels'])
    # break
    for it in list(labels.keys()):
        # print(it)
        for itm in labels[it]:
            if len(itm.keys())<3:
                
                print(th,':',i['labels'])
            # break

In [26]:
REdataset[18]

{'inputs': '黑客组织“AgainstTheWest”自2021年10月以来攻击了SonarQube、Gitblit和Gogs等平台，窃取了多家国内企事业单位的代码和数据，并在境外黑客论坛上非法售卖。共约150家单位受到影响，涵盖金融、医疗、政府、军事和高校等多个行业。该组织从受害单位窃取了大量信息系统源代码数据，并在境外黑客论坛RaidForums上进行非法售卖。',
 'targets': [['AgainstTheWest', 'Gogs', 'targets'],
  ['AgainstTheWest', 'SonarQube', 'targets'],
  ['AgainstTheWest', 'Gitblit', 'targets']],
 'labels': '{"(6, 11)": [{"(6, 11)": 4, "(22, 26)": 1, "relaid": 5}, {"(6, 11)": 4, "(27, 31)": 1, "relaid": 5}, {"(6, 11)": 4, "(32, 34)": 1, "relaid": 5}]}'}

In [27]:
REdataset = REdataset.remove_columns("targets")

In [28]:
REdataset

Dataset({
    features: ['inputs', 'labels'],
    num_rows: 741
})

#### 保存数据集
格式

    inputs:text
    labels:
    [
        subj1:{subj1,objs,rela},
        subj2:{subj2,objs,rela}
    ]

In [66]:
REdataset.save_to_disk("/home/RE/Dataset/pretrained_ds")

Saving the dataset (1/1 shards): 100%|██████████| 741/741 [00:00<00:00, 135435.74 examples/s]


#### 处理到Joint模式

    [
        1. subject 预测
        2. 根据 subject 预测object
    ]

In [12]:
from datasets import load_from_disk
import pandas
ds = load_from_disk('/home/RE/Dataset/pretrained_ds')
import torch
import collections
targets=[]
entity=0
for Ith,itm in enumerate(ds):
    dictone ={}
    inputs = tokenizer.encode_plus(itm["inputs"],padding=False,truncation=True)
    tokens=inputs['input_ids']
    
    mask=inputs['attention_mask']
    subj_type = collections.defaultdict(list)                       #创建空字典，但访问不存在的键只会返回空 list
    obj_type = collections.defaultdict(list)
    s1, s2 = [0] * len(tokens), [0] * len(tokens)
    items=collections.defaultdict(list)                             #   实体1 对应 实体2的字典
    itmx=[]
    labels = json.loads(itm['labels'])
    subject_index = labels.keys()
    for sub in subject_index:
        entity+=1
        dictone['subj']=[]
        dictone['obj']=[]
        s1[eval(sub)[0]]=labels[sub][0][sub]
        s2[eval(sub)[1]]=labels[sub][0][sub]
        
        o1, o2 = [0]*len(tokens), [0]*len(tokens)
        memory_inputs= [0]*len(tokens)
        
        memory_inputs[eval(sub)[0]:eval(sub)[1]]=[1]*(eval(sub)[1]-eval(sub)[0])
        for obj_rel in labels[sub]:
            obj_index = list(obj_rel.keys())[1]
            o1[eval(obj_index)[0]]=obj_rel['relaid']
            o2[eval(obj_index)[1]]=obj_rel['relaid']
        dictone['obj'].append({"obj_head":o1,"obj_tail":o2})
        dictone['subj'].append({"subj_head":subj_head,"subj_tail":subj_tail})
        
        
        targets.extend([{"inputs":tokens,"mask":mask,"subjs_head":s1,"subjs_tail":s2,'memory_input':memory_inputs,'objs_head':dictone['obj'][i]['obj_head'],"objs_tail":dictone['obj'][i]['obj_tail'],'targets':itm['targets']}for i,_ in enumerate(dictone['subj'])])
print(entity)
df= pandas.DataFrame(targets)

NameError: name 'subj_head' is not defined

In [14]:
from datasets import load_from_disk
import pandas
ds = load_from_disk('/home/RE/Dataset/pretrained_ds')
ds

Dataset({
    features: ['inputs', 'targets', 'labels'],
    num_rows: 741
})

In [78]:
dataset = Dataset.from_pandas(df)

In [79]:
dataset

Dataset({
    features: ['inputs', 'mask', 'subjs_head', 'subjs_tail', 'memory_input', 'objs_head', 'objs_tail', 'targets'],
    num_rows: 1256
})

#### 数据普遍长度3

In [33]:
max(len(i) for i in dataset['inputs'])

293

#### Change Pretrained Model to add_special_tokens

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/RE/LitTrain/PreModel/Bert-zh-en-cased")

In [18]:
print(tokenizer.encode('Today黑客组织“AgainstTheWest”自2021年10月以来攻击了SonarQube、Gitblit和Gogs等平台，窃取了多家国内企事业单位的代码和数据，并在境外黑客论坛上非法售卖。共约150家单位受到影响，涵盖金融、医疗、政府、军事和高校等多个行业。该组织从受害单位窃取了大量信息系统源代码数据，并在境外黑客论坛RaidForums上进行非法售卖。'))

[101, 100, 7946, 2145, 5299, 5302, 100, 100, 100, 5632, 9960, 2399, 8108, 3299, 809, 3341, 3122, 1140, 749, 100, 510, 100, 1469, 100, 5023, 2398, 1378, 8024, 4961, 1357, 749, 1914, 2157, 1744, 1079, 821, 752, 689, 1296, 855, 4638, 807, 4772, 1469, 3144, 2945, 8024, 2400, 1762, 1862, 1912, 7946, 2145, 6389, 1781, 677, 7478, 3791, 1545, 1297, 511, 1066, 5276, 8269, 2157, 1296, 855, 1358, 1168, 2512, 1510, 8024, 3891, 4667, 7032, 6084, 510, 1278, 4545, 510, 3124, 2424, 510, 1092, 752, 1469, 7770, 3413, 5023, 1914, 702, 6121, 689, 511, 6421, 5299, 5302, 794, 1358, 2154, 1296, 855, 4961, 1357, 749, 1920, 7030, 928, 2622, 5143, 5320, 3975, 807, 4772, 3144, 2945, 8024, 2400, 1762, 1862, 1912, 7946, 2145, 6389, 1781, 100, 677, 6822, 6121, 7478, 3791, 1545, 1297, 511, 102]


In [19]:
tokenizer.decode([101, 100, 7946, 2145, 5299, 5302, 100, 100, 100, 5632, 9960, 2399, 8108, 3299, 809, 3341, 3122, 1140, 749, 100, 510, 100, 1469, 100, 5023, 2398, 1378, 8024, 4961, 1357, 749, 1914, 2157, 1744, 1079, 821, 752, 689, 1296, 855, 4638, 807, 4772, 1469, 3144, 2945, 8024, 2400, 1762, 1862, 1912, 7946, 2145, 6389, 1781, 677, 7478, 3791, 1545, 1297, 511, 1066, 5276, 8269, 2157, 1296, 855, 1358, 1168, 2512, 1510, 8024, 3891, 4667, 7032, 6084, 510, 1278, 4545, 510, 3124, 2424, 510, 1092, 752, 1469, 7770, 3413, 5023, 1914, 702, 6121, 689, 511, 6421, 5299, 5302, 794, 1358, 2154, 1296, 855, 4961, 1357, 749, 1920, 7030, 928, 2622, 5143, 5320, 3975, 807, 4772, 3144, 2945, 8024, 2400, 1762, 1862, 1912, 7946, 2145, 6389, 1781, 100, 677, 6822, 6121, 7478, 3791, 1545, 1297, 511, 102])

'[CLS] [UNK] 黑 客 组 织 [UNK] [UNK] [UNK] 自 2021 年 10 月 以 来 攻 击 了 [UNK] 、 [UNK] 和 [UNK] 等 平 台 ， 窃 取 了 多 家 国 内 企 事 业 单 位 的 代 码 和 数 据 ， 并 在 境 外 黑 客 论 坛 上 非 法 售 卖 。 共 约 150 家 单 位 受 到 影 响 ， 涵 盖 金 融 、 医 疗 、 政 府 、 军 事 和 高 校 等 多 个 行 业 。 该 组 织 从 受 害 单 位 窃 取 了 大 量 信 息 系 统 源 代 码 数 据 ， 并 在 境 外 黑 客 论 坛 [UNK] 上 进 行 非 法 售 卖 。 [SEP]'

In [ ]:
import json, random, sys, collections, codecs
from torch.utils.data import Dataset, get_worker_info
from datasets import IterableDataset, load_dataset
import torch
from typing import Union, Dict
def process_func(itm, tokenizer, relatype) -> Dict:
    '''
    example : 
        context
        triplets:List[Dict[str,str]]
    '''
    
    encoding=tokenizer.encode_plus(itm["context"],padding=False,truncation=True,return_tensors="pt")
    
    tokens=encoding['input_ids']
    tokens=torch.squeeze(tokens, dim=0) 
    # 2. 掩码
    mask = encoding['attention_mask']# 数据掩码
    mask=torch.squeeze(mask, dim=0)

    
    # 记录s1,s2为实体下标位置
    # s1, s2 = [0] * len(tokens), [0] * len(tokens)
    rela=[[[0]*len(tokens)for _ in range(4)]for _ in  range(relatype)]                             #   关系字典
    for i,relapair in enumerate(itm['triple']):                     # 遍历该条记录中每个关系对
        itmx=list(relapair.keys())   
        if len(itmx)==2:
            # 记录首实体
            rela[relapair['relaid']-1][0][eval(itmx[0])[0]-1]=relapair[itmx[0]]
            rela[relapair['relaid']-1][1][eval(itmx[0])[1]-1]=relapair[itmx[0]]
            #记录尾实体
            rela[relapair['relaid']-1][2][eval(itmx[0])[0]-1]=relapair[itmx[0]]
            rela[relapair['relaid']-1][3][eval(itmx[0])[1]-1]=relapair[itmx[0]]
            

        else:
            rela[relapair['relaid']-1][0][eval(itmx[0])[0]-1]=relapair[itmx[0]]
            rela[relapair['relaid']-1][1][eval(itmx[0])[1]-1]=relapair[itmx[0]]
            rela[relapair['relaid']-1][2][eval(itmx[1])[0]-1]=relapair[itmx[1]]
            rela[relapair['relaid']-1][3][eval(itmx[1])[1]-1]=relapair[itmx[1]]
    relatensor = torch.tensor(rela)#25，4，len

    labelx=collections.defaultdict(list)
    for ith ,itm in enumerate(relatensor,start=1):
        labelx[f'{ith}']=itm.to_sparse_coo()
    return {"inputs":tokens,'mask':mask,"label":labelx}


## DataMatching&Padding


In [74]:
from collections import defaultdict
import pandas, random
from functools import partial
def seq_pad(seq, id, max_length):
    return seq+[id]*(max_length - len(seq))

def pad_sequence(elements, pad_id):
    batch = defaultdict(list)
    max_length = max([len(sample) for sample in elements['inputs']])
    print(max_length)
    Cus_seq_padding = partial(seq_pad,max_length = max_length)
    
    for element in elements:
        print(element)
        batch['inputs'].append(Cus_seq_padding(element['inputs'],id=pad_id))
        batch['masks'].append(Cus_seq_padding(element['mask'],id=0))
        batch['subjs_head'].append(Cus_seq_padding(element['subjs_head'], id=0))
        batch['subjs_tail'].append( Cus_seq_padding(element['subjs_tail'], id=0))
        batch['memory_input'].append(Cus_seq_padding(element['memory_input'], id =0))
        batch['objs_head'].append(Cus_seq_padding(element['obj_head'], id=0))
        batch['objs_tail'].append( Cus_seq_padding(element['obj_tail'],id =0))
    
    return {"inputs":torch.tensor(batch['inputs']),
            "masks":torch.tensor(batch["masks"]),
            "subjs_head":torch.tensor(batch['subjs_head']),
            "subjs_tail":torch.tensor(batch['subjs_tail']),
            "subj_head":torch.tensor(batch['subj_head']),
            "subj_tail":torch.tensor(batch['subj_tail']),
            "objs_head":torch.tensor(batch['objs_head']),
            "objs_tail":torch.tensor(batch['objs_tail']),
            "targets":batch['targets']}



In [ ]:
batchs=defaultdict(list)

for i in range(10):
    length = random.randint(3,10)
    batchs['inputs'].append([random.randint(1,5)]*length)
    batchs['mask'].append([1]*length)
    batchs['subjs_head'].append([1]*length)
    batchs['subjs_tail'].append([1]*length)
    batchs['subj_head'].append([i]*length)
    batchs['subj_tail'].append([i]*length)
    batchs['obj_head'].append([i]*length)
    batchs['obj_tail'].append([i]*length)
pd = pandas.DataFrame(batchs)

from datasets import load_dataset, Dataset
dataset = Dataset.from_pandas(pd)
print(dataset[3])
df=pad_sequence(dataset,100)

In [ ]:
df

In [32]:
model# 768

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(35225, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## Model setting
### A joint method for Information Extracting

#### subjlayer

In [ ]:
import torch
import torch.nn as nn
class SubTypeLayer(nn.Module):
    def __init__(self,args):
        super().__init__()
        
        self.layer_norm = nn.LayerNorm(args['hidden_size'])
        # layer_norm 
        self.mlphead = nn.Sequential(# 融合berthidden 和cnn 进行头部的判断
            nn.Linear(args['hidden_size'],256),
            nn.ReLU(),
            nn.Linear(256,args['num_subj_type'])
        )
        self.mlpend = nn.Sequential(
            nn.Linear(args['hidden_size'],256),
            nn.ReLU(),
            nn.Linear(256,args['num_subj_type'])
            )
        self.hiddenLinear=nn.Sequential(
            nn.Linear(args['hidden_size'],args['hidden_size']),
            nn.ReLU(),
            nn.Dropout(args['dropout'])
            )
        self.embLinear=nn.Sequential(
            nn.Linear(args['hidden_size'],768),
            nn.ReLU(),
            nn.Dropout(args['dropout'])
        )
        # 取消bert last_hidden_state的mlp层 
        self.last_stateLinear=nn.Sequential(
            nn.Linear(args['hidden_size'],768),
            nn.ReLU(),
            nn.Dropout(args['dropout'])
        )
        
    def forward(self,hidden,emb,last_state):
        # hidden=self.attention(hidden)
        hidden=self.hiddenLinear(hidden)
        emb=self.embLinear(emb)# 对emb内容进行考虑# 映射到 模型的隐藏层
        last_state=self.last_stateLinear(last_state)
        output1=torch.add(torch.add(hidden,emb),last_state)# 在seq_len上进行拼接
        output1 = self.layer_norm(output1)

        headlogits=self.mlphead(output1)#
        endlogits=self.mlpend(output1)
        return headlogits,endlogits

self.attention 可以观察隐藏层之间的状态关系
后续需要通过forward进行数据激活
Decoder
    weight = B,T,T matmul tril(T,T)
    tril
        [1,0,0,0]  考虑第一个元素
        [1,1,0,0]
        [1,1,1,0]
        

In [1]:
def compute_loss():
    '''
    compute_loss
    '''
    pass

In [17]:
import torch
import torch.nn.functional as F

# 创建一个3x3的张量作为示例输入
x = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

# 定义填充的方式，这里示例在四个方向上填充1个单位
padding = (2, 0)  # (left, right, top, bottom)

# 进行填充操作
padded_x = F.pad(x, padding, "constant", value=0)  # "constant"填充方式，填充值为0

print(padded_x)


tensor([[0, 0, 1, 2, 3],
        [0, 0, 4, 5, 6],
        [0, 0, 7, 8, 9]])


In [23]:
from torch.nn import Conv1d, functional
subj_model = Conv1d(768,768,3,1)

In [29]:
import torch.nn.functional as functional
inputs = tokenizer.encode_plus('Today黑客组织“AgainstTheWest”自2021年10月以来攻击了SonarQube、Gitblit和Gogs等平台，窃取了多家国内企事业单位的代码和数据，并在境外黑客论坛上非法售卖。共约150家单位受到影响，涵盖金融、医疗、政府、军事和高校等多个行业。该组织从受害单位窃取了大量信息系统源代码数据，并在境外黑客论坛RaidForums上进行非法售卖。',return_tensors='pt')
logits = model(**inputs)
channel_info = logits.last_hidden_state.permute(0,2,1)
logits = functional.pad(channel_info,(2,0),'constant',1)
# print(logits.last_hidden_state.shape)
logits = subj_model(logits).permute(0,2,1)
output = functional.max_pool1d(logits, kernel_size=3)
print(output.shape)

torch.Size([1, 150, 256])


#### Model Structure

In [30]:
from transformers import BertModel, BertForPreTraining, BertTokenizer
import torch.nn as nn


'''
Embedding +pos Embedding
        BERT
------------
      Extracter
      Subject --> Object
      Memory + Residual Link
'''
class Model(nn.Module):
    def __init__(self, model_path, **kwargs):
        super.__init__()
        self.tokenizer = BertTokenizer(model_path)
        self.base = BertModel(model_path)
        self.extracter = Extracter(**kwargs)
        
    def forward(self, inputs, memory_inputs = None):
        
        logits = self.base(inputs, mask)
        if memory_inputs is not None:
            subj_head_logits, subj_tail_logits, obj_head_logits, obj_tail_logits, _  = self.extracter(logits, memory_inputs)
        else:
            triples = self.extracter.predict(logits, inputs)
            return triples
        return subj_head_logits, subj_tail_logits, obj_head_logits, obj_tail_logits
    
class FeedForward(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
    def forward(self,hidden_states):
        return self.net(hidden_states)

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, **kwargs):
        super().__init__()
        self.key = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.query = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.value = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.soft_max = nn.Softmax()
    def forward(self, memory, hidden_states, memory_inputs):
        k = self.key(hidden_states)

        q = self.query(hidden_states)
        wei = q @ k.transpose(-2,-1) * self.hidden_dim**-0.5 # scaling attention
        wei = wei.masked_fill(memory !=0,float('-inf'))
        soft_wei = self.soft_max(wei) # 归1
        v = self.value(hidden_states)
        output = soft_wei @ v # B,L,L * B,L,dim =>B,L,dim
        return output
class Memory(nn.Module):#Memory + LCN
    def __init__(self, hidden_dim):
        super().__init__()
        self.self_attention = MultiHeadAttention(hidden_dim,head_nums =1)
        
        self.ffwd =FeedForward(hidden_dim=hidden_dim)
class MemoryUnit(nn.Module):
    def __init__(self, kwargs):
        pass
    
    def forward(self, hidden_state, memory_input):
        
        pass

class LCN(nn.Module):
    '''
    LCN 
    Joint a range of Hidden_states
    '''
    def __init__(self, **kwargs):
        self.LCNQ = nn.Conv1d(kwargs['hidden_size'], kwargs['hidden_size'] ,kwargs['kernel_size'], stride=1),
        self.LCNQn = nn.Sequential(
            nn.MaxPool1d(kernel_size=kwargs['kernel_size']),
            nn.Dropout(kwargs['dropout']),
            nn.ReLU()
        )
        self.ActiveLysH = nn.Sequential(
            nn.Linear(kwargs['hidden_size']/kwargs['kernel_size'],256),
            nn.ReLU(),
            nn.Linear(256,kwargs['output_layer'])
        )
        self.ActiveLysT = nn.Sequential(
            nn.Linear(kwargs['hidden_size']/kwargs['kernel_size'],256),
            nn.ReLU(),
            nn.Linear(256,kwargs['output_layer'])
        )
    def forward(self, inputs):
        
        inputs = functional.pad(inputs.permute(0, 2, 1), (2, 0), "constant", 0)
        LCN_logits = self.LCNQn(self.LCNQ(inputs).permute(0, 2, 1))
        headlogits = self.ActiveLysH(LCN_logits)
        taillogits = self.ActiveLysT(LCN_logits)
        return headlogits, taillogits
class Extracter(nn.Module): # --> hidden_states
    def __init__(self, **kwargs):
        self.subjextra = LCN(**kwargs)
        self.objextra = nn.ModuleDict(Memory(**kwargs))
    
    def forward(self, hidden_states, memory_inputs, subjs_head, subjs_tail, objs_head, objs_tail):
        '''
        
        '''
        subj_head_logits, subj_tail_logits = self.subjextra(hidden_states)
        obj_head_logits, obj_tail_logits = self.objextra(memory_inputs, hidden_states)
        subj_loss = compute_loss(subj_head_logits,subjs_head) + compute_loss(subj_tail_logits, subjs_tail)
        obj_loss = compute_loss(obj_head_logits, objs_head) + compute_loss(obj_tail_logits, objs_tail)
        loss = subj_loss + obj_loss
        return subj_head_logits, subj_tail_logits, obj_head_logits, obj_tail_logits, loss


    def predict(self, hidden_states, inputs):
        '''
        1. 获得主实体logits
        2. 区分主实体并获得其memory_inputs
        3. 获得每个主实体的关系Obj
        4. 返回主实体logits与objlogits
        '''
        
        subj_head_logits, subj_tail_logits = self.subjextra(hidden_states)
        triplelist = []
        memory_inputs = self.get_subjs(subj_head_logits, subj_tail_logits, inputs)# 
        for memory_input in memory_inputs:
            memoryinputs = torch.stack(torch.tensor(memory_input[0]),torch.tensor(memory_input[1]),dim=0)
            obj_head_logits, obj_tail_logits = self.objextra(memoryinputs, hidden_states)
            objs_infos = self.get_subjs(obj_head_logits, obj_tail_logits, inputs)#
            
            for itm in objs_infos:
                triplelist.append(self.tokenizer.decode(memory_input[2]),self.tokenizer.decode(itm[2]),itm[3])
            
        return set(triplelist)
    
    
    def get_subjs(self, startlogit , endlogit, inputs):
        def get_entity_indices(s1, s2, entity_type):
            start_indices = torch.nonzero(s1 == entity_type).squeeze(-1)
            end_indices = torch.nonzero(s2 == entity_type).squeeze(-1)
            entity_indices = []
            for start_index in start_indices:
                matching_end_indices = end_indices[end_indices >= start_index]  
                if len(matching_end_indices) > 0:
                    end_index = matching_end_indices[0].item()
                    entity_indices.append((start_index.item(), end_index))

            return entity_indices
        s1,s2 = startlogit.argmax(-1), endlogit.argmax(-1)#选择开始与结束位置  (batch,length)
        masked_s1 = torch.zeros_like(s1)
        masked_s1[mask.bool()] = s1[mask.bool()]
        masked_s2 = torch.zeros_like(s2)
        masked_s2[mask.bool()] = s2[mask.bool()]
        idlist=s1[(s1!=0)&(s1!=101)&(s1!=102)]
        subtensor=[]
        for value in torch.unique(idlist):
            entitylist=get_entity_indices(s1=s1,s2=s2,entity_type=value.item())
            for start,end in entitylist:
                tensor_start,tensor_tail = torch.zeros(len(inputs)), torch.zeros(len(inputs))
                tensor_start[start]=1
                tensor_tail[end]=1
                value_tensor = inputs[start:end+1]
                subtensor.append((tensor_start,tensor_tail,value_tensor,value.item()))
        return subtensor
        



In [ ]:
from transformers import AutoModelForTokenClassification,BertForTokenClassification

## Train

##### scheduler


In [31]:
from torch.optim import lr_scheduler
import torch.nn.functional as F
def get_scheduler(optimizer, scheduler_type, **kwargs):
    if scheduler_type == 'step':
        scheduler = lr_scheduler.StepLR(optimizer, step_size=kwargs['step_size'], gamma=kwargs['gamma'])
    elif scheduler_type == 'exp':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=kwargs['gamma']) # 不适宜太小
    elif scheduler_type == 'linear':
        scheduler = lr_scheduler.LinearLR(optimizer,start_factor=kwargs['start_factor'],end_factor=kwargs['end_factor'],total_iters= kwargs['total_iters'])
    elif scheduler_type == 'cosine':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=kwargs['T_max'],eta_min=kwargs['eta_min'])
    elif scheduler_type == 'reduce':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=kwargs['factor'], patience=kwargs['patience'])
    elif scheduler_type =="cosineAnneal":
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=kwargs['T_max'],eta_min=kwargs['eta_min'])
    else:
        scheduler = None
        print("Invalid scheduler type. Please choose from 'step', 'exp', or 'reduce'.")

    return scheduler

#### Optimizer

In [32]:
def get_optimizer(name, parameters,**kwargs):
    if name == 'sgd':
        return torch.optim.SGD(parameters, lr=kwargs['lr'])
    elif name == 'adam':
        return torch.optim.Adam(parameters, lr=kwargs['lr'], betas=(0.9, 0.99), **kwargs) 
    elif name == 'adamax':
        return torch.optim.Adamax(parameters, lr=kwargs['lr'])
    elif name == 'adamw':
        return torch.optim.AdamW(parameters,lr=kwargs['lr'])

    else:
        raise Exception("Unsupported optimizer: {}".format(name))

#### Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

from transformers import AdamW
import time
import wandb
from tqdm import tqdm
class ReTrainer(object):
    def __init__(self,trainingargs, modelargs,globalargs,trainloader,valloader,tokenizer,id2predicate,model):
        # 初始化
        # 学习率调度器初始化
        # optimizer初始化
        # dataloader
        # model
        # self.logger、helper
        # wandb.init(project="RETask",name=globalargs['wandbname'])
        self.device=trainingargs['device']
        self.trainloader=trainloader
        self.valloader = valloader
        self.model=model.to(self.device)
        self.trainingargs=trainingargs
        self.optimizer = get_optimizer(trainingargs['optim'],self.model.parameters(),**trainingargs)
        self.scheduler = get_scheduler(self.optimizer,trainingargs['scheduler'],**trainingargs)        
        self.globalargs = globalargs
        # globalargs config存放位置、model_save_dir:+id模型参数存放位置 \ +log表示模型日志文件存放位置
        if trainingargs['from_ckpt']:# 是否在检查点恢复训练
            try:
                self.load_model(trainingargs['from_ckpt'])
            except:
                FileNotFoundError
        self.num_epochs = trainingargs['num_epochs']
        self.tokenizer = tokenizer
        self.id2predicate = id2predicate
        self.checkpoints = globalargs['checkpoints'] if globalargs.get('checkpoints') else {}
        self.total_save_limit = trainingargs['total_save_limit']
        # self.optimizer=self.optimizer.to(self.device)
        pass
    # 损失函数计算方法
    def compute_loss(self,y_true,y_pred,mask):
        y_true=y_true.long()
        loss = F.cross_entropy(y_pred.reshape(-1,y_pred.size(-1)),y_true.reshape(-1))
        masked_loss = loss * mask
        masked_loss_value = torch.sum(masked_loss)/torch.sum(mask.float())
    
        return masked_loss_value
    def evaluate(self, elements):
        official_A, official_B, official_C = 1e-10, 1e-10, 1e-10
        
        results = []

        for batch in tqdm(iter(elements)):

            R = self.model(batch['inputs'])
            official_T = batch['targets']
            
            results.append({'text':tokenizer.decode(batch['inputs']), 'predict':list(R), 'truth':list(official_T)})
            # R 与official_T都是集合内容
            official_A += len(R & official_T)
            official_B += len(R)
            official_C += len(official_T)
        
        f1= 2* official_A/(official_B + official_C)
        prec = official_A/official_B if official_B>1 else 0
        recall = official_A/official_C
        return f1,prec,recall, results
    
    def train_iteration(self, trainingargs):
        # 对batch 进行一次前向传播
        # 将batch 投入到模型中进行前向传播

        global_steps=0
        global_start_time = time.time()
        dev_f1_history = []
        
        self.logger.log_train_start(Epochs=trainingargs['num_epochs'],lr=trainingargs['lr'])
        
        for epoch in tqdm(range(1,self.num_epochs+1)):
            train_loss =0
            
            for batch in tqdm(self.trainloader):
                start_time = time.time()
                global_steps+=1
                loss = self.train(batch)
                train_loss += loss
                if global_steps % trainingargs['log_steps'] == 0:
                    duration = time.time() - start_time
                    self.logger.log_epoch(epoch=epoch,loss= loss,num_epochs=self.num_epochs,duration=duration,lr=self.optimizer.param_groups[0]['lr'])
                    
                    wandb.log({"Loss":loss})
                if self.trainingargs.get('clip_grad_norm'):
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(),self.trainingargs['clip_grad_norm'])
                if global_steps % self.trainingargs['gradient_accumulation_steps']==0:
                    
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    self.model.zero_grad()
                    
                

            # -------------------------
            # 训练一个epoch 后
            
            self.logger.log_eval_start(len(self.valloader),epoch=epoch,num_epochs=self.num_epochs)
            
            dev_f1,dev_p,dev_r,results = self.evaluate(self.valloader, self.id2predicate, self.device)
            # 平均训练损失
            train_loss = train_loss /len(self.trainloader)
            # 判断是否为最优的f1值
            self.save_ckpt(epoch=epoch,performance_metric=dev_f1)
            # model_file = model_save_dir + '/ckpt_epoch_'
            if (epoch == 1 or dev_f1 >max(dev_f1_history) ) and trainingargs['predict']:
                best_f1 = dev_f1
                
                if not os.path.exists(self.globalargs['model_save_dir'] +'/predict'):
                    os.mkdir(self.globalargs['model_save_dir'] +'/predict')
                with open(self.globalargs['model_save_dir'] +'/predict'+ '/best_dev_results.json', 'w') as fw:
                    json.dump(results, fw, indent=4, ensure_ascii=False)
            elif epoch == 1 or dev_f1 >max(dev_f1_history):
                best_f1=dev_f1
            else:
                best_f1=max(dev_f1_history)       
            self.logger.log_evaluation(phase='evaluation',recall = dev_r,accuracy=dev_p,f1=dev_f1,best_f1=best_f1)
            dev_f1_history +=[dev_f1]
            #增加梯度裁剪 与 梯度步数累计
            self.scheduler.step()
            wandb.log({"Test Accu":dev_p,"Test F1":dev_f1,"Test Recall":dev_r})
        self.best_ckpt()
        wandb.finish()
        self.logger.log_train_end(self.num_epochs,best_f1,global_steps,duration=time.time()-global_start_time)
            
        pass
    def test(self,args):
            dev_f1,dev_p,dev_r,results = self.evaluate(self.valloader,self.tokenizer,self.id2predicate,self.device)
            # 平均训练损失
            # train_loss = train_loss /len(self.trainloader)
            # 判断是否为最优的f1值
            # self.save_ckpt(epoch=epoch,performance_metric=dev_f1)
            # model_file = model_save_dir + '/ckpt_epoch_'
            # if (epoch == 1 or dev_f1 >max(dev_f1_history) ) and args['predict']:
            #     best_f1 = dev_f1
            #     # copyfile(model_file, model_save_dir + '/best_model.pt')
            if not os.path.exists(self.globalargs['model_save_dir'] +'/predict'):
                os.mkdir(self.globalargs['model_save_dir'] +'/predict')
            with open(self.globalargs['model_save_dir'] +'/predict'+ '/best_dev_results.json', 'w') as fw:
                json.dump(results, fw, indent=4, ensure_ascii=False)
            # elif epoch == 1 or dev_f1 >max(dev_f1_history):
            #     best_f1=dev_f1
            # else:
            #     best_f1=max(dev_f1_history)       
            self.logger.log_evaluation(phase='test',recall = dev_r,accuracy=dev_p,f1=dev_f1,best_f1=None)
            

    def train(self,batch):
        inputs=batch["input_text"].to(self.device)
        
        mask=batch['mask'].to(self.device)
        memeoryinpus=batch['memory_input'].to(self.device)
        Subjs_head=batch["subjs_head"].to(self.device)
        Subjs_tail=batch["subjs_tail"].to(self.device)
        Objs_head=batch["objs_head"] .to(self.device)
        Objs_tail=batch["objs_tail"] .to(self.device)
        subMask = batch["subMask"].permute(1,0).to(self.device) # (subnums,batch)-->(batch,subnums)
        self.model.train()
        
        
        subj_head_logits, subj_tail_logits, obj_head_logits, obj_tail_logits = self.model(inputs=inputs, mask=mask,memory_inputs=memeoryinpus)
        subj_head_loss = self.compute_loss(Subjs_head, subj_head_logits, mask)
        subj_tail_loss = self.compute_loss(Subjs_tail, subj_tail_logits, mask)
        obj_head_loss = self.compute_loss(Subjs_head, subj_head_logits, mask)
        obj_tail_loss = self.compute_loss(Subjs_tail, subj_tail_logits, mask)
        
        loss=self.trainingargs['subj_loss_weight']*(subj_head_loss+subj_tail_loss)+self.trainingargs['type_loss_weight']*(obj_head_loss+obj_tail_loss)
        loss.backward()
        
        loss_val = loss.data.item()
        return loss_val
    
    def load_model(self,path):
        try:
            ckpt = torch.load(path)
            self.logger.load_from_ckpt(path)
        except:
            print('Error! files can not load')
            exit()
        self.model.load_state_dict(ckpt['model'])
        self.optimizer.load_state_dict = ckpt['optimizer_state_dict']
        self.scheduler.load_state_dict = ckpt['scheduler_state_dict']
        self.start_epoch = ckpt['epoch']+1
    def save_model(self, filename, epoch):
        params = {
                'model': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'epoch': epoch
                }
        try:
            torch.save(params, filename)
            self.logger.load_to_ckpt(filename)
        except BaseException:
            print("[Warning: Saving failed... continuing anyway.]")
    def save_ckpt(self, epoch, performance_metric):
        # 保存检查点
        if not os.path.exists(self.globalargs['model_save_dir']+'/'+'ckpt'):
            os.mkdir(self.globalargs['model_save_dir']+'/'+"ckpt")
        
        checkpoint_path = self.globalargs['model_save_dir'] +"/"+'ckpt'+'/'+f"checkpoint_{epoch}.pth"
        # ...保存模型状态、优化器状态等信息到checkpoint_path
        
        # 存储性能指标和检查点路径到字典中
        self.checkpoints[checkpoint_path] = performance_metric

        # 如果保存的检查点数量超过限制，则删除性能最差的检查点
        if len(self.checkpoints) > self.total_save_limit:
            # 找到性能最差的检查点
            # worst_checkpoint = min(self.checkpoints, key=lambda x:(self.checkpoints[x].get,self.checkpoints[x].split('.')[0].split('_')[-1]))
            worst_checkpoints = sorted(self.checkpoints.items(), key=lambda x:x[1])
            print(worst_checkpoints)
            # print(worst_checkpoint)
            worst_checkpoint = worst_checkpoints[0][0]
            for i in range(1,len(worst_checkpoints)):
                if self.checkpoints[worst_checkpoints[i][0]]>self.checkpoints[worst_checkpoint]:
                    break
                else:
                    if worst_checkpoint.split('_')[-1].split('.')[0] >worst_checkpoints[i][0].split('_')[-1].split('.')[0]:
                        worst_checkpoint=worst_checkpoints[i][0]
                    else:
                        continue

            if os.path.exists(worst_checkpoint):
                os.remove(worst_checkpoint)
                # print(os.listdir('/'.join(worst_checkpoint.split('/')[:-1])))
                del self.checkpoints[worst_checkpoint]  # 从字典中删除最差的检查点
                self.save_model(checkpoint_path,epoch=epoch)
            else:
                del self.checkpoints[worst_checkpoint]
        else:
            self.save_model(checkpoint_path,epoch=epoch)
    def best_ckpt(self):
        best_checkpoint = max(self.checkpoints, key=self.checkpoints.get)
        copyfile(best_checkpoint, self.globalargs['model_save_dir'] + '/best_model.pt')


In [ ]:
from torch.utils.data import DataLoader, random_split
from datasets import load_dataset, load_from_disk
import wandb
def workflow(batch_size,data_path,val_size,tokenizer, task_name):
    all_dataset = load_from_disk('/home/RE/Dataset/pretrained_ds')
    
    traindataset,valdataset = all_dataset,None if val_size <0.0001 else random_split(all_dataset,[len(all_dataset)-int(len(all_dataset)*val_size),int(len(all_dataset)*val_size)])
    my_data_collator = partial(pad_sequence, pad_id = tokenizer.pad_token_id)

    trainloader = DataLoader(traindataset, batch_size= batch_size, shuffle = True, collate_fn = my_data_collator)
    if valdataset is not None:
        valloader = DataLoader(valdataset, batch_size = batch_size, shuffle = True, collate_fn = my_data_collator)
    wandb.init(project=task_name, name="wandb"+f"size_{val_size}_batch{batch_size}")
    wandb.watch(model)
    trainer = ReTrainer(
        trainloader = trainloader,
        valloader=valloader,
        model = model,
        tokenizer = tokenizer,
        
    )
    


## hyperparams
1. data_path
2. model_layers

In [ ]:
from datasets import Dataset
class Seq2SeqDataset(Dataset):
    pass


## Score test bench